In [1]:
import os
import string
import re
from nltk.stem import *
import nltk
import re
import math

In [2]:
# Global variables
folder_path = 'articles_all'
file_name = "collated_data.txt"
file_contents_list = []


file_name_number_mapping_dict = {}
number_file_name_mapping_dict = {}
inverted_index_intermediate = {}
inverted_index = {}
num_docs = 1
idf_dict = {}
tf_dict = {}
file_to_vec_dict = {}
query_to_vec_dict = {}

In [3]:
class fileContents:
    file_name = ""
    file_number = 0
    contents_raw = ""
    contents_tokens = []

In [4]:
# check
def tokenize_and_remove_punctuations(s):
    translator = str.maketrans('','',string.punctuation)
    modified_string = s.translate(translator)
    modified_string = ''.join([i for i in modified_string if not i.isdigit()])
    return nltk.word_tokenize(modified_string)

In [5]:
def get_stopwords():
    stop_words = [word for word in open('stopwords.txt','r').read().split('\n')]
    return stop_words

In [6]:
def parse_data(contents):
    contents = contents.lower()
    title_start = contents.find('<title>')
    title_end = contents.find('</title>')
    title = contents[title_start+len('<title>'):title_end]
    text_start = contents.find('<text>')
    text_end = contents.find('</text>')
    text = contents[text_start+len('<text>'):text_end]
    return title+" "+text

In [7]:
# check
def stem_words(tokens):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(token) for token in tokens]
    return stemmed_words

In [8]:
# check
def remove_stop_words(tokens):
    stop_words = get_stopwords()
    filtered_words = [token for token in tokens if token not in stop_words and len(token) > 2]
    return filtered_words

In [10]:
def get_vocabulary(data):
    tokens = []
    for token_list in data.values():
        tokens = tokens + token_list
    fdist = nltk.FreqDist(tokens)
    return list(fdist.keys())

In [11]:
# check
def preprocess_data(contents):
    dataDict = {}
    data_list = []
    for content in contents:

        tokens = tokenize_and_remove_punctuations(content)
        filtered_tokens = remove_stop_words(tokens)
        stemmed_tokens = stem_words(filtered_tokens)

        if len(stemmed_tokens) !=  0:
            data_list.append(stemmed_tokens)

    return data_list

In [13]:
# dont delete
def read_data(path):
    curr_file_num = 1
    
    for filename in os.listdir(path):
        # print (filename)
        if curr_file_num%1000 == 0:
            print (curr_file_num)

        global file_contents_list

        file_data = fileContents()
        
        data = parse_data(open(path + '/' + filename,'r').read())
    
        data = data.replace('\n', ' ').replace("'''", '').replace("''", '').replace(",", ' ').strip()

        # print (data)
        file_data.contents_raw = data
        file_data.file_num = curr_file_num
        file_data.file_name = filename

        data_list = data.split()
        file_data.contents_tokens = preprocess_data(data_list)

        file_contents_list.append(file_data)

        # inverted_index = generate_inverted_index(preprocess_data_dict)
        curr_file_num += 1
    return  inverted_index 

In [19]:
#Read tokensied data from file 
with open(file_name, 'r') as file:
    lines = file.readlines()

for line in lines:
    content = line.split("!@#")
    file_content = fileContents()
    file_content.file_name = content[0]
    file_content.file_number = int(content[1])
    file_content.contents_raw = content[2]
    file_content.contents_tokens = content[3].replace("'], ['", ",").replace("\n", "").strip("'[]'").split(',')

    file_contents_list.append(file_content)
    

In [20]:
# Create inverted index

for entry in file_contents_list:
    for token in entry.contents_tokens:
        if token in inverted_index_intermediate.keys():
            inverted_index_intermediate[token].append(entry.file_number)
        else:
            inverted_index_intermediate[token] = [entry.file_number]


for key, val in inverted_index_intermediate.items():
    temp_dict = {}
    # print (key, val)
    
    for num in val:
        if num in temp_dict:
            temp_dict[num] += 1
        else:
            temp_dict[num] = 1
    

    for key1, val1 in temp_dict.items():
        if len(key) > 1:
            if key in inverted_index:
                inverted_index[key].append([key1, val1])
            else:
                inverted_index[key] = [[key1, val1]]
    

with open('inverted_index.txt', 'w') as file:
    for key, val in inverted_index.items():

        line = key + '\t' + str(val) + '\n'
        file.write(line) 

